In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import os, os.path 
import numpy
import pickle
from glob import glob
from typing import Any, Dict, List, Tuple, Union
import pandas as pd

"""Change to the data folder"""
new_path = "./new_train/new_train"
test_path = './new_val_in/new_val_in'
subset_test_path = './new_train/train_subset'
# number of sequences in each dataset
# train:205942  val:3200 test: 36272 
# sequences sampled at 10HZ rate

### Create a dataset class 

In [2]:
class ArgoverseDataset(Dataset):
    """Dataset class for Argoverse"""
    def __init__(self, data_path: str, transform=None):
        super(ArgoverseDataset, self).__init__()
        self.data_path = data_path
        self.transform = transform

        self.pkl_list = glob(os.path.join(self.data_path, '*'))
        self.pkl_list.sort()
        
    def __len__(self):
        return len(self.pkl_list)

    def __getitem__(self, idx):

        pkl_path = self.pkl_list[idx]
        with open(pkl_path, 'rb') as f:
            data = pickle.load(f)
            
        if self.transform:
            data = self.transform(data)

        return data


# intialize a dataset
val_dataset  = ArgoverseDataset(data_path=new_path)
test_dataset = ArgoverseDataset(data_path=test_path)

### Create a loader to enable batch processing

In [3]:
batch_sz = 6

def train_collate(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """
    inp = [numpy.dstack([scene['p_in'][scene['track_id'][:,0,0]==scene['agent_id'],:,:]]) for scene in batch]
    out = [numpy.dstack([scene['p_out'][scene['track_id'][:,0,0]==scene['agent_id'],:,:]]) for scene in batch]
    inp = torch.Tensor(inp)
    out = torch.Tensor(out)
    return [inp, out]

def test_collate(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """
    inp = [numpy.dstack([scene['p_in'][scene['track_id'][:,0,0]==scene['agent_id'],:,:]]) for scene in batch]
    inp = torch.Tensor(inp)
    idx = [numpy.dstack([scene['scene_idx']]) for scene in batch]
    return inp, idx
    
val_loader = DataLoader(val_dataset,batch_size=batch_sz, shuffle = True, collate_fn=train_collate, num_workers=0)

test_loader = DataLoader(test_dataset,batch_size=batch_sz, shuffle = True, collate_fn=test_collate, num_workers=0)

In [4]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class EncoderRNN(nn.Module):
    """referenced from official Argoverse forecasting code: https://github.com/jagjeet-singh/argoverse-forecasting"""
    
    def __init__(self,
                 input_size = 2,
                 embedding_size = 8,
                 hidden_size = 16):
        
        super(EncoderRNN, self).__init__()
        
        self.hidden_size = hidden_size
        self.linear = nn.Linear(input_size, embedding_size)
        self.lstm = nn.LSTMCell(embedding_size, hidden_size)

    def forward(self, x, hidden):
        embedded = F.relu(self.linear(x))
        hidden = self.lstm(embedded, hidden)
        return hidden


class DecoderRNN(nn.Module):
    """Decoder Network."""
    """referenced from official Argoverse forecasting code: https://github.com/jagjeet-singh/argoverse-forecasting"""
    def __init__(self, embedding_size=8, hidden_size=16, output_size=2):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.linear1 = nn.Linear(output_size, embedding_size)
        self.lstm = nn.LSTMCell(embedding_size, hidden_size)
        self.linear2 = nn.Linear(hidden_size, output_size)

    def forward(self, x, hidden):
        embedded = F.relu(self.linear1(x))
        hidden = self.lstm(embedded, hidden)
        output = self.linear2(hidden[0])
        return output, hidden


In [5]:
from tqdm import tqdm_notebook as tqdm

def train(encoder, decoder, device, train_loader, encoder_optimizer, decoder_optimizer, epoch, log_interval=10000):    
    """referenced from official Argoverse forecasting code: https://github.com/jagjeet-singh/argoverse-forecasting"""
    
    iterator = tqdm(train_loader, total=int(len(train_loader)))
    counter = 0
    criterion = nn.MSELoss()
    
    for i_batch, sample_batch in enumerate(train_loader):
        
        inp, out = sample_batch
        # preprocessing more ????
        inp = inp[:,0,:,:]
        out = out[:,0,:,:]
        
        #inp - inp[0] for all in whaetver
        x_offset = []
        y_offset = []
        for i in range(inp.shape[0]):
            x_offset.append(inp[i][0][0].detach().clone())
            y_offset.append(inp[i][0][1].detach().clone())
    
        for j in range(inp.shape[0]):
            for i in range(inp.shape[1]):
                inp[j][i][0] = inp[j][i][0] - x_offset[j]
                inp[j][i][1] = inp[j][i][1] - y_offset[j]

        #outoput whatever
        for j in range(out.shape[0]):
            for i in range(out.shape[1]):
                out[j][i][0] = out[j][i][0] - x_offset[j]
                out[j][i][1] = out[j][i][1] - y_offset[j]
        
        _input, target = inp.to(device), out.to(device)
        
        encoder.train()
        decoder.train()
        
        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()
        
        
        #encoder 
        batch_size = _input.shape[0]
        input_length = _input.shape[1]
        output_length = target.shape[1]
        feature_len = _input.shape[2]
        input_shape = _input.shape[2]
        
        encoder_hidden = (torch.zeros(batch_size, encoder.module.hidden_size).to(device), 
                          torch.zeros(batch_size, encoder.module.hidden_size).to(device))
        
        loss = 0
        
        # Encode observed trajectory
        for ei in range(input_length):
            encoder_input = _input[:, ei, :]
            encoder_hidden = encoder(encoder_input, encoder_hidden)

        # Initialize decoder input with last coordinate in encoder
        decoder_input = encoder_input[:, :2]

        # Initialize decoder hidden state as encoder hidden state
        decoder_hidden = encoder_hidden

        decoder_outputs = torch.zeros(target.shape).to(device)

        # Decode hidden state in future trajectory
        for di in range(30):
            decoder_output, decoder_hidden = decoder(decoder_input,
                                                     decoder_hidden)
            decoder_outputs[:, di, :] = decoder_output

            # Update loss
            loss += torch.sqrt(criterion(decoder_output[:, :2], target[:, di, :2]))

            # Use own predictions as inputs at next step
            decoder_input = decoder_output

        # Get average loss for pred_len
        loss = loss / 30

        # Backpropagate
        loss.backward()
        encoder_optimizer.step()
        decoder_optimizer.step()
        
        file1 = open("loss_steps.txt", "a")  # append mode
        file1.write(str(loss.item()) + ",")
        file1.close()
        
#       output = model(data)
#         loss = MSELoss(output, target)
        counter += 1
        iterator.set_postfix(loss=(loss.item()))
    

In [6]:
device = "cuda"
encoder = EncoderRNN(input_size=2)
decoder = DecoderRNN(output_size=2)

encoder = nn.DataParallel(encoder)
decoder = nn.DataParallel(decoder)

encoder.to(device)
decoder.to(device)

encoder_optimizer = torch.optim.Adam(encoder.parameters())
decoder_optimizer = torch.optim.Adam(decoder.parameters())

num_epoch = 16

for epoch in range(1, num_epoch + 1):
        train(encoder, decoder, device, val_loader, encoder_optimizer, decoder_optimizer, epoch)
#         predict(model, device, test_loader)

In [7]:
def infer_absolute(
        test_loader: torch.utils.data.DataLoader,
        encoder: EncoderRNN,
        decoder: DecoderRNN,
#         start_idx: int,
#         forecasted_save_dir: str,
#         model_utils: ModelUtils,
):
    """Infer function for non-map LSTM baselines and save the forecasted trajectories.
    
    referenced from official Argoverse forecasting code: https://github.com/jagjeet-singh/argoverse-forecasting
    
    Args:
        test_loader: DataLoader for the test set
        encoder: Encoder network instance
        decoder: Decoder network instance
        start_idx: start index for the current joblib batch
        forecasted_save_dir: Directory where forecasted trajectories are to be saved
        model_utils: ModelUtils instance

    """
    
    forecasted_trajectories = {}

    for i, (_input, idx) in enumerate(test_loader):
        
        _input = _input[:,0,:,:]
        
        #inp - inp[0] for all in whaetver
        x_offset = []
        y_offset = []
        for i in range(_input.shape[0]):
            x_offset.append(_input[i][0][0].detach().clone())
            y_offset.append(_input[i][0][1].detach().clone())
    
        for j in range(_input.shape[0]):
            for i in range(_input.shape[1]):
                _input[j][i][0] = _input[j][i][0] - x_offset[j]
                _input[j][i][1] = _input[j][i][1] - y_offset[j]

        _input = _input.to(device)

        # Set to eval mode
        encoder.eval()
        decoder.eval()

        # Encoder
        batch_size = _input.shape[0]
        input_length = _input.shape[1]
        input_shape = _input.shape[2]

        # Initialize encoder hidden state
        encoder_hidden = (torch.zeros(batch_size, encoder.module.hidden_size).to(device), 
                          torch.zeros(batch_size, encoder.module.hidden_size).to(device))
       
        # Encode observed trajectory
        for ei in range(input_length):
            encoder_input = _input[:, ei, :]
            encoder_hidden = encoder(encoder_input, encoder_hidden)

        # Initialize decoder input with last coordinate in encoder
        decoder_input = encoder_input[:, :2]

        # Initialize decoder hidden state as encoder hidden state
        decoder_hidden = encoder_hidden

        decoder_outputs = torch.zeros(
            (batch_size, 30, 2)).to(device)

        # Decode hidden state in future trajectory
        for di in range(30):
            decoder_output, decoder_hidden = decoder(decoder_input,
                                                     decoder_hidden)
            decoder_outputs[:, di, :] = decoder_output

            # Use own predictions as inputs at next step
            decoder_input = decoder_output

        for i in range(30):
            for j in range(batch_size):
                decoder_outputs[j,i,0] = decoder_outputs[j,i,0] + x_offset[j]
                decoder_outputs[j,i,1] = decoder_outputs[j,i,1] + y_offset[j]
            
                if (idx[j][0][0][0] in forecasted_trajectories):
                    forecasted_trajectories[idx[j][0][0][0]].append(decoder_outputs[j,i,:].tolist())
                else:
                    forecasted_trajectories[idx[j][0][0][0]] = [decoder_outputs[j,i,:].tolist()]
                
    return(forecasted_trajectories)

In [8]:
output = infer_absolute(test_loader, encoder, decoder)

In [9]:
import pandas as pd
df = pd.DataFrame.from_dict(output, orient='index')
df

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
23825,"[2140.90478515625, 688.7164916992188]","[2141.717041015625, 689.3495483398438]","[2142.246826171875, 689.9172973632812]","[2143.046875, 690.5863037109375]","[2143.830078125, 691.2218627929688]","[2144.59326171875, 691.8628540039062]","[2145.3271484375, 692.499755859375]","[2146.036376953125, 693.1346435546875]","[2146.72607421875, 693.765869140625]","[2147.401611328125, 694.3927001953125]",...,"[2154.83447265625, 701.0126953125]","[2155.52587890625, 701.5977783203125]","[2156.215087890625, 702.1807861328125]","[2156.900146484375, 702.7615356445312]","[2157.580078125, 703.3397827148438]","[2158.253173828125, 703.9151611328125]","[2158.91845703125, 704.4871826171875]","[2159.574951171875, 705.055419921875]","[2160.221435546875, 705.6192016601562]","[2160.857421875, 706.177734375]"
31653,"[2138.53955078125, 807.7974853515625]","[2138.864501953125, 807.5703735351562]","[2138.94287109375, 807.0562133789062]","[2139.0390625, 806.7268676757812]","[2139.13134765625, 806.3141479492188]","[2139.26708984375, 805.9163208007812]","[2139.42724609375, 805.4974365234375]","[2139.61572265625, 805.0711669921875]","[2139.826904296875, 804.635986328125]","[2140.05712890625, 804.1951293945312]",...,"[2142.99609375, 799.1886596679688]","[2143.2568359375, 798.7197875976562]","[2143.513671875, 798.248291015625]","[2143.766357421875, 797.7743530273438]","[2144.01513671875, 797.29833984375]","[2144.259521484375, 796.8206787109375]","[2144.5, 796.3419799804688]","[2144.736572265625, 795.8629150390625]","[2144.96923828125, 795.3840942382812]","[2145.197998046875, 794.9063720703125]"
37433,"[668.2675170898438, 1389.4259033203125]","[667.6193237304688, 1389.2850341796875]","[666.9008178710938, 1389.20703125]","[666.1661987304688, 1389.308837890625]","[665.509033203125, 1389.3095703125]","[664.83544921875, 1389.317626953125]","[664.1615600585938, 1389.308349609375]","[663.4850463867188, 1389.2862548828125]","[662.8078002929688, 1389.253173828125]","[662.1311645507812, 1389.2132568359375]",...,"[654.9102172851562, 1388.877197265625]","[654.2694702148438, 1388.8663330078125]","[653.6287841796875, 1388.8568115234375]","[652.9873657226562, 1388.84765625]","[652.3446655273438, 1388.8380126953125]","[651.6998901367188, 1388.8270263671875]","[651.0523681640625, 1388.813720703125]","[650.401611328125, 1388.79736328125]","[649.7469482421875, 1388.777099609375]","[649.0880126953125, 1388.752197265625]"
33230,"[1722.64892578125, 559.5056762695312]","[1723.35546875, 560.1295776367188]","[1723.8714599609375, 560.6915283203125]","[1724.62744140625, 561.3316040039062]","[1725.361083984375, 561.9393310546875]","[1726.070068359375, 562.5499267578125]","[1726.747802734375, 563.1546630859375]","[1727.39990234375, 563.7550048828125]","[1728.03125, 564.3492431640625]","[1728.6474609375, 564.9366455078125]",...,"[1735.4049072265625, 571.0296630859375]","[1736.043212890625, 571.5645141601562]","[1736.682373046875, 572.097900390625]","[1737.32080078125, 572.6299438476562]","[1737.9573974609375, 573.1607055664062]","[1738.590576171875, 573.6901245117188]","[1739.2193603515625, 574.2181396484375]","[1739.842529296875, 574.7444458007812]","[1740.4591064453125, 575.2688598632812]","[1741.0682373046875, 575.791015625]"
7167,"[2013.06591796875, 592.6577758789062]","[2011.9361572265625, 591.9609375]","[2010.9300537109375, 591.001708984375]","[2009.9755859375, 590.143310546875]","[2008.9964599609375, 589.268310546875]","[2008.0018310546875, 588.388671875]","[2006.995849609375, 587.5039672851562]","[2005.984130859375, 586.61767578125]","[2004.9700927734375, 585.7319946289062]","[2003.9554443359375, 584.8480224609375]",...,"[1992.8856201171875, 575.2410278320312]","[1991.8917236328125, 574.3770141601562]","[1990.9014892578125, 573.5162353515625]","[1989.9158935546875, 572.65966796875]","[1988.9359130859375, 571.80810546875]","[1987.962646484375, 570.962646484375]","[1986.9970703125, 570.124267578125]","[1986.040771484375, 569.293

In [10]:
for i in range(30):
    df[['v{}'.format((i*2)+1), 'v{}'.format((i*2)+2)]] = pd.DataFrame(df.get(i).tolist(), index=df.index)

In [11]:
import numpy as np
dropped_cols = list(np.arange(30))
df2 = df.drop(dropped_cols, axis=1)
df2.index.name = 'ID'
df2

,v1,v2,v3,v4,v5,v6,v7,v8,v9,v10,...,v51,v52,v53,v54,v55,v56,v57,v58,v59,v60
ID,,,,,,,,,,,,,,,,,,,,,
23825,2140.904785,688.716492,2141.717041,689.349548,2142.246826,689.917297,2143.046875,690.586304,2143.830078,691.221863,...,2158.253174,703.915161,2158.918457,704.487183,2159.574951,705.055420,2160.221436,705.619202,2160.857422,706.177734
31653,2138.539551,807.797485,2138.864502,807.570374,2138.942871,807.056213,2139.039062,806.726868,2139.131348,806.314148,...,2144.259521,796.820679,2144.500000,796.341980,2144.736572,795.862915,2144.969238,795.384094,2145.197998,794.906372
37433,668.267517,1389.425903,667.619324,1389.285034,666.900818,1389.207031,666.166199,1389.308838,665.509033,1389.309570,...,651.699890,1388.827026,651.052368,1388.813721,650.401611,1388.797363,649.746948,1388.777100,649.088013,1388.752197
33230,1722.648926,559.505676,1723.355469,560.129578,1723.871460,560.691528,1724.627441,561.331604,1725.361084,561.939331,...,1738.590576,573.690125,1739.219360,574.218140,1739.842529,574.744446,1740.459106,575.268860,1741.068237,575.791016
7167,2013.065918,592.657776,2011.936157,591.960938,2010.930054,591.001709,2009.975586,590.143311,2008.996460,589.268311,...,1987.962646,570.962646,1986.997070,570.124268,1986.040771,569.293762,1985.094604,568.472046,1984.160278,567.659912
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34644,201.765991,1523.023682,201.901291,1523.015381,202.591461,1522.931519,203.139984,1523.069824,203.584045,1523.140869,...,211.840637,1525.066406,212.151779,1525.122559,212.450851,1525.172852,212.737457,1525.217163,213.011383,1525.255249
29931,2137.809570,810.362305,2137.994873,810.064209,2138.080811,810.047729,2138.127930,809.952148,2138.258789,809.819946,...,2143.859375,804.604431,2144.097900,804.402283,2144.336182,804.215454,2144.574707,804.045532,2144.814453,803.894287
39464,2017.450073,591.914185,2018.940186,592.944580,2019.999878,594.015198,2021.248047,594.948303,2022.481201,595.884277,...,2047.343994,617.563538,2048.363525,618.401428,2049.341309,619.183105,2050.276123,619.913391,2051.166504,620.598022


In [12]:
df2.to_csv("outputs16ep_b6.csv", index=True)